# 466 final project

### What we want 

we want to be able to take a set of users and use their previous transactions to predict what dapps they will use in the future.

## Load and clean user account data
these users will be split up into training and testing sets
this list of users was pulled from pontem's space pirate whitelist.
Pontem is a dapp that allows users to earn 'interest' on their crypto holdings by providing liquidity through their dex (sets of liquidity pools).

In [3]:
import pandas as pd
#load the users csv file (
users = open('users.csv').readlines()
user_temp = []
for user in users:
    if len(user)>=66:
        user_temp.append(user.split(',')[0])
print("ALL user ADDR COUNT:",len(users))
users = user_temp

print("PROPPER ADDR COUNT:",len(users))

# Write the users to a new file
# with open('users_clean.csv', 'w') as f:
#     for user in users:
#         f.write(user)


ALL user ADDR COUNT: 31622
PROPPER ADDR COUNT: 31622


We also need to load in the information about the dapps that we want to predict for. We will use the dapp list created by 'seam-money' an explorer for dapps on the network

In [4]:
dapps = open('dapps.csv').readlines()
# print(dapps)
dapp_dic = {}
dapp_types = {}
for dapp in dapps:
    d = dapp.split(',')
    dapp_dic[d[0]] = d[1]
    dapp_types[d[1]] = dapp_types.get(d[-1])
print("DAPP COUNT:",len(dapp_dic))
print("DAPP TYPES:",len(dapp_types))
print(dapp_dic)


DAPP COUNT: 14
{'0x34e2eead0aefbc3d0af13c0522be94b002658f4bef8e0740a21086d22236ad77': 'Switchboard', '0x62fdfe47c9c37227be1f885e79be827be292fe1833ac63a2fe2c2c16c55ecb12': 'KanaLabs', '0xd1fd99c1944b84d1670a2536417e997864ad12303d19eac725891691b04d614e': 'BlueMove', '0x89576037b3cc0b89645ea393a47787bb348272c76d6941c574b053672b848039': 'Hippo Labs', '0x2c7bccf7b31baf770fdbcc768d9e9cb3d87805e255355df5db32ac9a669010a2': 'Souffl3', '0x9770fa9c725cbd97eb50b2be5f7416efdfd1f1554beb0750d4dae4c64e860da3': 'Aries', '0x190d44266241744264b964a37b8f09863167a12d3e70cda39376cfb4e3561e12': 'liquidSwap', '0x16fe2df00ea7dde4a63409201f7f4e536bde7bb7335526a35d05111e68aa322c': 'AnimeSwap', '0xa61f1c8f0e69bb866f2073f9a1ab0252efdd9f12c092256c68290d8dae35551e': 'Mojito', '0xd4c70815e9d245a097646af821ecf87a073039e71e93c8cd04a0da082134d296': 'Topaz', '0x98298d34bcf896c663e069c464754e0cfd36b50e21eedd8db0e4189168057cb7': 'Argo', '0x7ab72b249ec24f76fe66b6de19dcee1e3d3361db5c2cccfaa48ea8659060a1bd': 'HoustonSwap', '0

In [5]:
users[0]

'0x9ee9892d8600ed0bf65173d801ab75204a16ac2c6f190454a3b98f6bcb99d915'

In [18]:
# from flask import jsonify
import requests
import random
NODE_URL = "https://fullnode.mainnet.aptoslabs.com/v1"
def get_account_transactions(address):
    user_dapps = {}
    response = requests.get(
        f"{NODE_URL}/accounts/{address}/transactions",
        params={"limit": 50}
        )
    txs = response.json()
    # print(len(txs))
    txns_temp = []
    for tx in txs:
        try:
            txn = {}
            payload = tx['payload']
            to = payload['function'].split('::')[0]
            func = payload['function'].split('::')[2]
            mod = payload['function'].split('::')[1]
            txn['function'] = func
            txn['module'] = mod
            if to in dapp_dic.keys():
                dapp_name = dapp_dic[to]
                if dapp_name in user_dapps.keys():
                    user_dapps[dapp_name] = 1 + user_dapps[dapp_name]
                else:
                    user_dapps[dapp_name] = 1
            txns_temp.append(txn)
        except:
            # print(e)
            pass


    return txns_temp,user_dapps


def find_dapps_used(address):
    transactions,dapps = get_account_transactions(address)
    return dapps


users_used_dapps = {}
demo_users = random.sample(users, 100)
for user in users[100:110]:
    # print("USER:",user)
    dapps = find_dapps_used(user)
    if len(dapps.keys())>1:
        users_used_dapps[user] = dapps
        print(users_used_dapps[user])




{'Souffl3': 8, 'liquidSwap': 10, 'Hippo Labs': 2}
{'liquidSwap': 4, 'Souffl3': 5}
{'liquidSwap': 2, 'Souffl3': 3}
{'liquidSwap': 2, 'BlueMove': 6}
{'liquidSwap': 15, 'Souffl3': 5, 'Topaz': 1}


In [22]:
# find the average number of dapps used by a user
# find the average number of transactions per dapp

def avg_dapps_used(users_used_dapps):
    total = 0
    for user in users_used_dapps.keys():
        total += len(users_used_dapps[user].keys())
    return total/len(users_used_dapps.keys())

avg_dapps_by_user = avg_dapps_used(users_used_dapps)
print("AVG DAPPS USED BY USER:",avg_dapps_by_user)



2.4


In [24]:
def avg_txns_per_dapp(users_used_dapps):
    totals = {}
    for user in users_used_dapps.keys():
        for dapp in users_used_dapps[user].keys():
            if dapp in totals.keys():
                totals[dapp] += users_used_dapps[user][dapp]
            else:
                totals[dapp] = users_used_dapps[user][dapp]
    print(totals)
    return totals

totals = avg_txns_per_dapp(users_used_dapps)

{'Souffl3': 21, 'liquidSwap': 33, 'Hippo Labs': 2, 'BlueMove': 6, 'Topaz': 1}


In [20]:
#  use k-means to cluster the users based on the dapps they use
#  use the clusters to find the most popular dapps in each cluster
dapp_ls = dapp_dic.values()
def dapp_diff(user1,user2):
    diff_dic = {}
    for dapp in dapp_ls:
        if dapp in user1.keys() and dapp in user2.keys():
            diff_dic[dapp] = abs(user1[dapp]-user2[dapp])
        elif dapp in user1.keys() and dapp not in user2.keys():
            diff_dic[dapp] = user1[dapp]
        elif dapp not in user1.keys() and dapp in user2.keys():
            diff_dic[dapp] = user2[dapp]
        else:
            pass
    print(diff_dic)
    return diff_dic
user_addrs = list(users_used_dapps.keys())
user1 = users_used_dapps[user_addrs[0]]
print(user1)
user2 = users_used_dapps[user_addrs[1]]
print(user2)
dapp_diff(user1,user2)


{'Souffl3': 8, 'liquidSwap': 10, 'Hippo Labs': 2}
{'liquidSwap': 4, 'Souffl3': 5}
{'Hippo Labs': 2, 'Souffl3': 3, 'liquidSwap': 6}


{'Hippo Labs': 2, 'Souffl3': 3, 'liquidSwap': 6}

In [ ]:
# categorize the dapps by type

In [23]:
# use dapp_diff to compute the distance between each user
def user_distance(user1,user2):
    diff_dic = dapp_diff(user1,user2)
    total = 0
    for dapp in diff_dic.keys():
        total += diff_dic[dapp]
    return total
d = user_distance(user1,user2)
print(d)

{'Hippo Labs': 2, 'Souffl3': 3, 'liquidSwap': 6}
11


In [8]:
FLASK_URL = "localhost:5000"